# Install Dependencies

In [ ]:
%pip install --upgrade pip pandas scikit-learn scipy

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
from scipy.stats import randint, uniform
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [ ]:
RANDOM_STATE = 33

# Load Data

In [ ]:
dataset_df = pd.read_csv('../kaggle/datasets/spaceship-titanic/train.csv')
print("Train dataset shape:", dataset_df.shape)

In [ ]:
# Extract the target variable
y = dataset_df["Transported"]
X = dataset_df.drop(["Transported"], axis=1)

In [ ]:
# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

The data is also available in the [Kaggle Spaceship Titanic competition](https://www.kaggle.com/competitions/spaceship-titanic/data).

# Exploritory Data Analysis

**train.csv** - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
- `PassengerId` - A unique Id for each passenger. Each Id takes the form `gggg_pp` where `gggg` indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
- `HomePlanet` - The planet the passenger departed from, typically their planet of permanent residence.
- `CryoSleep` - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
- `Cabin` - The cabin number where the passenger is staying. Takes the form `deck/num/side`, where side can be either P for Port or S for Starboard.
- `Destination` - The planet the passenger will be debarking to.
- `Age` - The age of the passenger.
- `VIP` - Whether the passenger has paid for special VIP service during the voyage.
- `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
- `Name` - The first and last names of the passenger.
- `Transported` - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

**test.csv** - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of Transported for the passengers in this set.

**sample_submission.csv** - A submission file in the correct format.
- `PassengerId` - Id for each passenger in the test set.
- `Transported` - The target. For each passenger, predict either True or False.

In [ ]:
dataset_df.info()

In [ ]:
dataset_df.head()

Looking at the data in combination with the description, we can see that the data is a mix of categorical and numerical data. The categorical data is `PassengerId`, `HomePlanet`, `CryoSleep`, `Cabin`, `Destination`, `VIP`, `Name`, and `Transported`. The numerical data is `Age`, `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, and `VRDeck`.

The description reveals further information which is not immediately obvious and can be used to engineer new features. The `PassengerId` is a unique identifier for each passenger, but it is also a group identifier. The `Cabin` column contains information about the deck, room number, and side of the ship.

In [ ]:
dataset_df.describe()

The descriptive statistics reveal that the most of the passengers are in their 20s and 30s, with a mean age of 28.82. Half of the passengers do not have any charges for the amenities. 

In [ ]:
# Check for missing values
dataset_df.isnull().sum()

All columns have around 200 missing values (except for the `PassengerId` and the Target `Transported`) which is around 2% of the total dataset.

In [ ]:
# Check rows without any missing values
dataset_df.dropna().shape

If we drop the rows with missing values, we will lose around 25% of the data. This is a significant amount of data to lose, so we will need to impute the missing values. It tells us that the missing values are spread across the dataset and not concentrated in a few rows.

# Custom Preprocessing

While sci-kit learn has a lot of preprocessing tools, some of the preprocessing steps are too specific to the dataset to be included in the library. For example, the `Cabin` column contains information about the deck, room number, and side of the ship. We can extract this information and create new features.

In [ ]:
class PassengerIdSplitter(BaseEstimator, TransformerMixin):
    """Split the PassengerId into Group and Number"""
    
    def fit(self, X: pd.DataFrame, y=None):
        return self

    def transform(self, X: pd.DataFrame):
        # Split the PassengerId into Group and Number
        X["Group"] = X["PassengerId"].str.split("_").str[0]
        X["Number"] = X["PassengerId"].str.split("_").str[1]
        # Drop the original column
        return X.drop(["PassengerId"], axis=1)
        

In [ ]:
class CabinSplitter(BaseEstimator, TransformerMixin):
    """Split the Cabin into Deck and Room"""
    
    def fit(self, X: pd.DataFrame, y=None):
        return self
    
    def transform(self, X: pd.DataFrame):
        # Split the Cabin into Deck, Room and Side (port or starboard)
        X["Deck"] = X["Cabin"].str.split("/").str[0]
        X["Room"] = X["Cabin"].str.split("/").str[1]
        X["Side"] = X["Cabin"].str.split("/").str[2]
        # Drop the original column
        return X.drop(["Cabin"], axis=1)

In [ ]:
class ColumnDropper(BaseEstimator, TransformerMixin):
    """Drop the specified columns"""

    def __init__(self, columns):
        self.columns = columns

    def fit(self, X: pd.DataFrame, y=None):
        return self

    def transform(self, X: pd.DataFrame):
        # Drop the specified columns
        return X.drop(self.columns, axis=1)

# Column Transformer

Now that we have our custom preprocessing steps, we can create a column transformer. This will allow us to apply different preprocessing steps to different columns based on their data type.

First, we will create a pipeline for the numerical data. We will use the `SimpleImputer` to impute the missing values with the median. Then we will use the `StandardScaler` to scale the data.

In [ ]:
numerical_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

Next, we will create a pipeline for the categorical data. We will use the `SimpleImputer` to impute the missing values with the most frequent value. Then we will use the `OneHotEncoder` to encode the categorical data. We will use the `handle_unknown='ignore'` parameter to ignore unknown categories in the test set and the `sparse=False` parameter to return a full array instead of a sparse matrix.

In [ ]:
categorical_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])


Finally, we will combine the two pipelines using the `ColumnTransformer`. Here we can use the `make_column_selector` to select the columns we want to apply the specific pipeline to. This works because we kept all numerical columns which represent categorical data as strings.

In [ ]:
column_transformer = ColumnTransformer([
    ('numerical_preprocessing', numerical_preprocessor, make_column_selector(dtype_include=np.number)),
    ('categorical_preprocessing', categorical_preprocessor, make_column_selector(dtype_include=object))
])

# Creating a Baseline Model

Now that we have our data preprocessing steps, we can create a baseline model. We will use the Random Forest Classifier with default hyperparameters as our baseline model.

In [ ]:
pipeline = Pipeline([
    ('column_dropper', ColumnDropper(columns=["Name"])),
    ('column_transformer', column_transformer),
    ('classifier', GradientBoostingClassifier(random_state=RANDOM_STATE))
    ])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
accuracy_score = pipeline.score(X_test, y_test)
print(f"Accuracy score: {accuracy_score:.3}")

# Model Selection and Hyperparameter Tuning

Now that we have a baseline model, we can try different models and tune the hyperparameters to improve the model performance. We will use the Random Forest Classifier,  AdaBoost and GradientBoosting. We will use the `RandomizedSearchCV` to tune the hyperparameters for each model.

In [ ]:
param_distributions = {
    "classifier__n_estimators": randint(100, 1000),
    "classifier__learning_rate": uniform(0.01, 0.3),
    "classifier__max_depth": randint(3,16),
    'min_samples_split': randint(2, 100),  
    'min_samples_leaf': randint(1, 50),
    'max_features': ['sqrt', 'log2', None],
}

In [ ]:
random_search = RandomizedSearchCV(
    pipeline, 
    param_distributions,
    scoring="accuracy",
    n_iter=10,
    cv=3, 
    verbose=1, 
    n_jobs=-1
)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
best_estimator = random_search.best_estimator_
print(best_estimator)

In [ ]:
best_params = random_search.best_params_
print(best_params)

In [ ]:
random_search.score(X_test, y_test)
print(f"Accuracy score: {accuracy_score:.3}")